In [51]:
import pandas as pd
import numpy as np
import chess.pgn
import chess
from random import randint
from random import shuffle
import tensorflow as tf
from tensorflow import keras
from keras import layers, callbacks, losses
from keras import regularizers
from keras import backend as K

In [52]:
def fen2bitstring(fen):
    board = chess.Board(fen)
    bitboard = np.zeros(64*6*2+5)

    piece_idx = {'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5}

    for i in range(64):
        if board.piece_at(i):
            color = int(board.piece_at(i).color) + 1
            bitboard[(piece_idx[board.piece_at(i).symbol().lower()] + i * 6) * color] = 1

    bitboard[-1] = int(board.turn)
    bitboard[-2] = int(board.has_kingside_castling_rights(True))
    bitboard[-3] = int(board.has_kingside_castling_rights(False))
    bitboard[-4] = int(board.has_queenside_castling_rights(True))
    bitboard[-5] = int(board.has_queenside_castling_rights(False))

    return bitboard
    

In [53]:
def randPosiGeneratorFromGame(game):
    random_positions_array = []
    positions = []
    board = game.board()

    for move in game.mainline_moves():
        if not board.is_capture(move):
            position = board.fen()
            bitstring_position = fen2bitstring(position)
            positions.append(bitstring_position)

        board.push(move)

        no_capture_posis = len(positions)
    indices = list(range(5, no_capture_posis))
    shuffle(indices)
    selected_indices = indices[:15]

    random_positions_array = [positions[index] for index in selected_indices]

    return random_positions_array

In [54]:
pgn_file = open("CCRLdb.pgn")

random_positions_white_win_bitstring = []
random_positions_black_win_bitstring = []

i = 0
while i < (10000):
    game = chess.pgn.read_game(pgn_file)
    if game.headers["Result"] == "1-0":
        temp1 = randPosiGeneratorFromGame(game)
        random_positions_white_win_bitstring += temp1
    elif game.headers["Result"] == "0-1":
        temp2 = randPosiGeneratorFromGame(game)
        random_positions_black_win_bitstring += temp2
    if game is None:
        break
    i = i + 1
    print(f"{i}\t")
pgn_file.close()

1	
2	
3	
4	
5	
6	
7	
8	
9	
10	
11	
12	
13	
14	
15	
16	
17	
18	
19	
20	
21	
22	
23	
24	
25	
26	
27	
28	
29	
30	
31	
32	
33	
34	
35	
36	
37	
38	
39	
40	
41	
42	
43	
44	
45	
46	
47	
48	
49	
50	
51	
52	
53	
54	
55	
56	
57	
58	
59	
60	
61	
62	
63	
64	
65	
66	
67	
68	
69	
70	
71	
72	
73	
74	
75	
76	
77	
78	
79	
80	
81	
82	
83	
84	
85	
86	
87	
88	
89	
90	
91	
92	
93	
94	
95	
96	
97	
98	
99	
100	
101	
102	
103	
104	
105	
106	
107	
108	
109	
110	
111	
112	
113	
114	
115	
116	
117	
118	
119	
120	
121	
122	
123	
124	
125	
126	
127	
128	
129	
130	
131	
132	
133	
134	
135	
136	
137	
138	
139	
140	
141	
142	
143	
144	
145	
146	
147	
148	
149	
150	
151	
152	
153	
154	
155	
156	
157	
158	
159	
160	
161	
162	
163	
164	
165	
166	
167	
168	
169	
170	
171	
172	
173	
174	
175	
176	
177	
178	
179	
180	
181	
182	
183	
184	
185	
186	
187	
188	
189	
190	
191	
192	
193	
194	
195	
196	
197	
198	
199	
200	
201	
202	
203	
204	
205	
206	
207	
208	
209	
210	
211	
212	
213	
214	
215	
216	
217	
218	
219	
220	
221	
222

In [55]:
len(random_positions_white_win_bitstring)

51540

In [56]:
min(len(random_positions_black_win_bitstring), len(random_positions_white_win_bitstring))

40845

In [57]:
x = 0
y = min(len(random_positions_black_win_bitstring), len(random_positions_white_win_bitstring))//3
w = min(len(random_positions_black_win_bitstring), len(random_positions_white_win_bitstring))//3
t = min(len(random_positions_black_win_bitstring), len(random_positions_white_win_bitstring))

In [58]:
white_win_bitstring_tensor = tf.convert_to_tensor(random_positions_white_win_bitstring[x:y], dtype=tf.float32)

black_win_bitstring_tensor = tf.convert_to_tensor(random_positions_black_win_bitstring[x:y], dtype=tf.float32)


In [59]:
white_win_dataset = tf.data.Dataset.from_tensor_slices(white_win_bitstring_tensor)
black_win_dataset = tf.data.Dataset.from_tensor_slices(black_win_bitstring_tensor)

In [60]:
concatenated_dataset = white_win_dataset.concatenate(black_win_dataset)

In [61]:
num_samples = len(concatenated_dataset)
train_dataset = concatenated_dataset.shuffle(num_samples)
print('Train Dataset Size:', len(train_dataset))

Train Dataset Size: 27230


In [62]:
train_data = np.array(list(train_dataset.as_numpy_iterator()))

In [64]:
def scheduler1(epoch, lr):
    return lr * 0.98

# Create the learning rate scheduler callback
lr_scheduler = callbacks.LearningRateScheduler(scheduler1)


In [65]:
optimiser1=tf.keras.optimizers.Adam(lr = 0.005)

In [66]:
def train_autoencoders(train_data):  #[[e1, d1], [c1], [e2, d2], [c2], [e3, d3], [c3], [e4, d4], [o]]
    weights = []
    # Layer 1: 773-600-773 Autoencoder

    input1 = layers.Input(shape=(773,))
    dropout1 = layers.Dropout(0.25)(input1)
    encoder1 = layers.Dense(600, activation='relu')(dropout1)
    decoder1 = layers.Dense(773, activation='relu')(encoder1)
    layer1 = tf.keras.Model(inputs=input1, outputs=decoder1)
    layer1.compile(optimizer=optimiser1, loss = losses.MeanSquaredError())
    K.set_value(layer1.optimizer.learning_rate, 0.005)
    layer1.fit(train_data, train_data, epochs=200, batch_size = 500, callbacks=[lr_scheduler])
    weights.append(layer1.layers[2].get_weights())
    
    
    # Layer 2: 600-400-600 Autoencoder
    
    input2 = layers.Input(shape=(773,))
    dropout2 = layers.Dropout(0.25)(input2)
    l1_2 = layers.Dense(600, activation = 'relu')(dropout2)
    encoder2 = layers.Dense(400, activation='relu')(l1_2)
    decoder2 = layers.Dense(600, activation='relu')(encoder2)
    decoder2_1 = layers.Dense(773, activation = 'relu')(decoder2)
    layer2 = tf.keras.Model(inputs=input2, outputs=decoder2_1)
    layer2.layers[2].set_weights(weights[0])
    layer2.compile(optimizer=optimiser1, loss = losses.MeanSquaredError())
    K.set_value(layer2.optimizer.learning_rate, 0.005)
    layer2.fit(train_data, train_data, epochs=200, batch_size = 500, callbacks=[lr_scheduler])
    weights.append(layer2.layers[3].get_weights())
    

    # Layer 3: 400-200-400 Autoencoder

    input3 = layers.Input(shape=(773,))
    dropout3 = layers.Dropout(0.25)(input3)
    l1_3 = layers.Dense(600, activation = 'relu')(dropout3)
    l2_3 = layers.Dense(400, activation = 'relu')(l1_3)
    encoder3 = layers.Dense(200, activation='relu')(l2_3)
    decoder3 = layers.Dense(400, activation='relu')(encoder3)
    decoder3_1 = layers.Dense(600, activation = 'relu')(decoder3)
    decoder3_2 = layers.Dense(773, activation = 'relu')(decoder3_1)
    layer3 = tf.keras.Model(inputs=input3, outputs=decoder3_2)
    layer3.layers[2].set_weights(weights[0])
    layer3.layers[3].set_weights(weights[1])
    layer3.compile(optimizer=optimiser1, loss = losses.MeanSquaredError())
    K.set_value(layer3.optimizer.learning_rate, 0.005)
    layer3.fit(train_data, train_data, epochs=200, batch_size = 500, callbacks=[lr_scheduler])
    weights.append(layer3.layers[4].get_weights())
    

    # Layer 4: 200-100-200 Autoencoder

    input4 = layers.Input(shape=(773,))
    dropout4 = layers.Dropout(0.25)(input4)
    l1_4  =layers.Dense(600, activation = 'relu')(dropout4)
    l2_4 = layers.Dense(400, activation = 'relu')(l1_4)
    l3_4 = layers.Dense(200, activation = 'relu')(l2_4)
    encoder4 = layers.Dense(100, activation='relu')(l3_4)
    decoder4 = layers.Dense(200, activation='relu')(encoder4)
    decoder4_1 = layers.Dense(400, activation = 'relu')(decoder4)
    decoder4_2 = layers.Dense(600, activation = 'relu')(decoder4_1)
    decoder4_3 = layers.Dense(773, activation = 'relu')(decoder4_2)
    layer4 = tf.keras.Model(inputs=input4, outputs=decoder4_3)
    layer4.layers[2].set_weights(weights[0])
    layer4.layers[3].set_weights(weights[1])
    layer4.layers[4].set_weights(weights[2])
    layer4.compile(optimizer=optimiser1, loss = losses.MeanSquaredError())
    K.set_value(layer4.optimizer.learning_rate, 0.005)
    layer4.fit(train_data, train_data, epochs=200, batch_size = 500, callbacks=[lr_scheduler])
    weights.append(layer4.layers[5].get_weights())
    
    return weights

In [67]:
pos2vec_weights = train_autoencoders(train_data)


Epoch 1/200
55/55 [==============================] - 1s 14ms/step - loss: 0.0125 - lr: 0.0049
Epoch 2/200
55/55 [==============================] - 1s 14ms/step - loss: 0.0095 - lr: 0.0048
Epoch 3/200
55/55 [==============================] - 1s 14ms/step - loss: 0.0093 - lr: 0.0047
Epoch 4/200
55/55 [==============================] - 1s 13ms/step - loss: 0.0092 - lr: 0.0046
Epoch 5/200
55/55 [==============================] - 1s 13ms/step - loss: 0.0091 - lr: 0.0045
Epoch 6/200
55/55 [==============================] - 1s 14ms/step - loss: 0.0090 - lr: 0.0044
Epoch 7/200
55/55 [==============================] - 1s 12ms/step - loss: 0.0089 - lr: 0.0043
Epoch 8/200
55/55 [==============================] - 1s 13ms/step - loss: 0.0088 - lr: 0.0043
Epoch 9/200
55/55 [==============================] - 1s 12ms/step - loss: 0.0087 - lr: 0.0042
Epoch 10/200
55/55 [==============================] - 1s 13ms/step - loss: 0.0087 - lr: 0.0041
Epoch 11/200
55/55 [==============================] - 1s 13

Epoch 1/200
55/55 [==============================] - 1s 22ms/step - loss: 0.0150 - lr: 0.0049
Epoch 2/200
55/55 [==============================] - 1s 21ms/step - loss: 0.0096 - lr: 0.0048
Epoch 3/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0088 - lr: 0.0047
Epoch 4/200
55/55 [==============================] - 1s 19ms/step - loss: 0.0084 - lr: 0.0046
Epoch 5/200
55/55 [==============================] - 1s 19ms/step - loss: 0.0083 - lr: 0.0045
Epoch 6/200
55/55 [==============================] - 1s 19ms/step - loss: 0.0082 - lr: 0.0044
Epoch 7/200
55/55 [==============================] - 1s 18ms/step - loss: 0.0081 - lr: 0.0043
Epoch 8/200
55/55 [==============================] - 1s 22ms/step - loss: 0.0080 - lr: 0.0043
Epoch 9/200
55/55 [==============================] - 1s 24ms/step - loss: 0.0080 - lr: 0.0042
Epoch 10/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0080 - lr: 0.0041
Epoch 11/200
55/55 [==============================] - 1s 19

Epoch 1/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0183 - lr: 0.0049
Epoch 2/200
55/55 [==============================] - 1s 21ms/step - loss: 0.0135 - lr: 0.0048
Epoch 3/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0119 - lr: 0.0047
Epoch 4/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0112 - lr: 0.0046
Epoch 5/200
55/55 [==============================] - 1s 21ms/step - loss: 0.0108 - lr: 0.0045
Epoch 6/200
55/55 [==============================] - 1s 21ms/step - loss: 0.0105 - lr: 0.0044
Epoch 7/200
55/55 [==============================] - 1s 21ms/step - loss: 0.0104 - lr: 0.0043
Epoch 8/200
55/55 [==============================] - 1s 21ms/step - loss: 0.0102 - lr: 0.0043
Epoch 9/200
55/55 [==============================] - 1s 21ms/step - loss: 0.0100 - lr: 0.0042
Epoch 10/200
55/55 [==============================] - 1s 21ms/step - loss: 0.0099 - lr: 0.0041
Epoch 11/200
55/55 [==============================] - 1s 20

Epoch 1/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0185 - lr: 0.0049
Epoch 2/200
55/55 [==============================] - 1s 19ms/step - loss: 0.0153 - lr: 0.0048
Epoch 3/200
55/55 [==============================] - 1s 21ms/step - loss: 0.0139 - lr: 0.0047
Epoch 4/200
55/55 [==============================] - 1s 22ms/step - loss: 0.0131 - lr: 0.0046
Epoch 5/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0125 - lr: 0.0045
Epoch 6/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0121 - lr: 0.0044
Epoch 7/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0117 - lr: 0.0043
Epoch 8/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0115 - lr: 0.0043
Epoch 9/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0113 - lr: 0.0042
Epoch 10/200
55/55 [==============================] - 1s 20ms/step - loss: 0.0111 - lr: 0.0041
Epoch 11/200
55/55 [==============================] - 1s 21

In [68]:
weights_encoder1 = pos2vec_weights[0]
weights_encoder2 = pos2vec_weights[1]
weights_encoder3 = pos2vec_weights[2]
weights_encoder4 = pos2vec_weights[3]

In [69]:
# Weights are taken after training on pos2vec first
def create_pos2vec_model():
    input_layer = layers.Input(shape = (773,))
    dropouti = layers.Dropout(0.25)(input_layer)
    x1 = layers.Dense(600, activation='relu')(dropouti)
    dropout1 = layers.Dropout(0.25)(x1)
    x2 = layers.Dense(400, activation='relu')(dropout1)
    dropout2 = layers.Dropout(0.25)(x2)
    x3 = layers.Dense(200, activation='relu')(dropout2)
    dropout3 = layers.Dropout(0.25)(x3)
    x4 = layers.Dense(100, activation='relu')(dropout3)
    
    dbn_model = tf.keras.Model(inputs=input_layer, outputs=x4)
    
    # Set the weights for the DBN layers
    dbn_model.layers[2].set_weights(weights_encoder1)  # First encoder layer
    dbn_model.layers[4].set_weights(weights_encoder2)  # Second encoder layer
    dbn_model.layers[6].set_weights(weights_encoder3)  # Third encoder layer
    dbn_model.layers[8].set_weights(weights_encoder4)  # Fourth encoder layer
    #dbn_model.trainable = False

    return dbn_model

In [70]:
def create_siamese_network():
    dbn = create_pos2vec_model()
    
    input1 = layers.Input(shape = (773,))
    input2 = layers.Input(shape = (773,))
    
    features1 = dbn(input1)
    features2 = dbn(input2)
    
    concatenated_features = layers.Concatenate()([features1, features2])
    
    fc1 = layers.Dense(400, activation = 'relu')(concatenated_features)
    dropout1 = layers.Dropout(0.25)(fc1)
    fc2 = layers.Dense(200, activation = 'relu')(dropout1)
    dropout2 = layers.Dropout(0.25)(fc2)
    fc3 = layers.Dense(100, activation = 'relu')(dropout2)
    #fc4 = layers.Dense(50, activation = 'relu')(fc3)
    #fc5 = layers.Dense(25, activation = 'relu')(fc4)
    #fc6 = layers.Dense(12, activation = 'relu')(fc5)
    output_layer = layers.Dense(2, activation = 'softmax')(fc3)
    
    siamese_model = tf.keras.Model(inputs=[input1, input2], outputs=output_layer)
    
    return siamese_model

In [71]:
win = [1]
white_win_labels = []
for i in range(len(random_positions_white_win_bitstring)):
    white_win_labels.append(win)
loss = [0]
black_win_labels = []
for i in range(len(random_positions_black_win_bitstring)):
    black_win_labels.append(loss)

In [72]:
white_win_bitstring_tensor = tf.convert_to_tensor(random_positions_white_win_bitstring[w:t], dtype=tf.float32)

white_win_labels_tensor = tf.convert_to_tensor(white_win_labels[w:t], dtype=tf.int32)

black_win_bitstring_tensor = tf.convert_to_tensor(random_positions_black_win_bitstring[w:t], dtype=tf.float32)

black_win_labels_tensor = tf.convert_to_tensor(black_win_labels[w:t], dtype=tf.int32)

In [73]:
white_win_dataset = tf.data.Dataset.from_tensor_slices(white_win_bitstring_tensor)
black_win_dataset = tf.data.Dataset.from_tensor_slices(black_win_bitstring_tensor)

white_win_labels_dataset = tf.data.Dataset.from_tensor_slices(white_win_labels_tensor)
black_win_labels_dataset = tf.data.Dataset.from_tensor_slices(black_win_labels_tensor)

In [74]:
paired_dataset = tf.data.Dataset.zip((white_win_dataset, black_win_dataset))
paired_labels_dataset = tf.data.Dataset.zip((white_win_labels_dataset, black_win_labels_dataset))

In [75]:
len(paired_dataset)

27230

In [76]:
from sklearn.model_selection import train_test_split

# Convert ZipDataset to numpy arrays
paired_dataset_array = np.array(list(paired_dataset.as_numpy_iterator()))
paired_labels_dataset_array = np.array(list(paired_labels_dataset.as_numpy_iterator()))
# Split ratio for train-test split
split_ratio = 0.8

# Split the paired dataset and labels dataset into train and test
train_dataset, test_dataset, train_labels_dataset, test_labels_dataset = train_test_split(
    paired_dataset_array, paired_labels_dataset_array, test_size=1-split_ratio)

# Print the sizes of train and test datasets
print('Train Dataset Size:', len(train_dataset))
print('Train Labels Dataset Size:', len(train_labels_dataset))
print('Test Dataset Size:', len(test_dataset))
print('Test Labels Dataset Size:', len(test_labels_dataset))

Train Dataset Size: 21784
Train Labels Dataset Size: 21784
Test Dataset Size: 5446
Test Labels Dataset Size: 5446


In [77]:
for i in range(len(train_dataset)//2):
    train_dataset[2*i][0], train_dataset[2*i][1] = train_dataset[2*i][1].copy(), train_dataset[2*i][0].copy()
    train_labels_dataset[2*i][0], train_labels_dataset[2*i][1] = train_labels_dataset[2*i][1].copy(), train_labels_dataset[2*i][0].copy()

for i in range(len(test_dataset) // 2):
    test_dataset[2*i][0], test_dataset[2*i][1] = test_dataset[2*i][1].copy(), test_dataset[2*i][0].copy()
    test_labels_dataset[2*i][0], test_labels_dataset[2*i][1] = test_labels_dataset[2*i][1].copy(), test_labels_dataset[2*i][0].copy()

In [78]:
def scheduler2(epoch, lr):
    return lr * 0.99

# Create the learning rate scheduler callback
lr_scheduler = callbacks.LearningRateScheduler(scheduler2)

In [79]:
optimiser2 = tf.keras.optimizers.Adam(lr=0.01)


In [80]:
siamese_model = create_siamese_network()
# Extract inputs from the paired_dataset_array
paired_dataset_input1 = train_dataset[:, 0]
paired_dataset_input2 = train_dataset[:, 1]

# Create and compile the Siamese model
siamese_model.compile(optimizer=optimiser2, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['categorical_accuracy'])
K.set_value(siamese_model.optimizer.learning_rate, 0.01)
siamese_model.fit([paired_dataset_input1, paired_dataset_input2], train_labels_dataset, epochs=1000, batch_size=500, callbacks = [lr_scheduler], validation_data=([test_dataset[:, 0], test_dataset[:, 1]], test_labels_dataset), validation_batch_size=500, validation_freq=10)

Epoch 1/1000
44/44 [==============================] - 1s 18ms/step - loss: 0.7150 - categorical_accuracy: 0.5237 - lr: 0.0099
Epoch 2/1000
44/44 [==============================] - 1s 19ms/step - loss: 0.5867 - categorical_accuracy: 0.6858 - lr: 0.0098
Epoch 3/1000
44/44 [==============================] - 1s 20ms/step - loss: 0.5053 - categorical_accuracy: 0.7475 - lr: 0.0097
Epoch 4/1000
44/44 [==============================] - 1s 23ms/step - loss: 0.4475 - categorical_accuracy: 0.7842 - lr: 0.0096
Epoch 5/1000
44/44 [==============================] - 1s 21ms/step - loss: 0.4065 - categorical_accuracy: 0.8124 - lr: 0.0095
Epoch 6/1000
44/44 [==============================] - 1s 22ms/step - loss: 0.3639 - categorical_accuracy: 0.8314 - lr: 0.0094
Epoch 7/1000
44/44 [==============================] - 1s 20ms/step - loss: 0.3364 - categorical_accuracy: 0.8447 - lr: 0.0093
Epoch 8/1000
44/44 [==============================] - 1s 20ms/step - loss: 0.3133 - categorical_accuracy: 0.8580 - lr:

In [81]:
# Evaluate the Siamese model on the test dataset
test_loss, test_accuracy = siamese_model.evaluate([test_dataset[:, 0], test_dataset[:, 1]], test_labels_dataset)

# Print the accuracy
print('Test Accuracy:', test_accuracy*100)

171/171 [==============================] - 0s 2ms/step - loss: 0.1968 - categorical_accuracy: 0.9152
Test Accuracy: 91.51670932769775


In [82]:
# Evaluate the Siamese model on the test dataset
test_loss, test_accuracy = siamese_model.evaluate([paired_dataset_input1, paired_dataset_input2], train_labels_dataset)

# Print the accuracy
print('Test Accuracy:', test_accuracy*100)

681/681 [==============================] - 1s 1ms/step - loss: 0.0149 - categorical_accuracy: 0.9975
Test Accuracy: 99.75211024284363


In [83]:
def preprocess_for_siamese_input(fen1, fen2):
    bitstring1 = np.array([fen2bitstring(fen1)])
    bitstring2 = np.array([fen2bitstring(fen2)])
    siamese_input = [bitstring1, bitstring2]
    return siamese_input

In [84]:
fen1 = "3r3k/p4pq1/7Q/4pB2/7P/2N5/PPP2P2/6RK b - - 0 34"  #position with queen blunder
fen2 = "3r3k/p4pQ1/7p/4pB2/7P/2N5/PPP2P2/6RK b - - 0 34"   #best move acc. to stockfish

fin1 = preprocess_for_siamese_input(fen1, fen2)
x = siamese_model.predict(fin1)
x

1/1 [==============================] - 0s 61ms/step


array([[0.20680003, 0.7932    ]], dtype=float32)

In [85]:
fin2 = preprocess_for_siamese_input(fen2, fen1)
y = siamese_model.predict(fin2)
y

1/1 [==============================] - 0s 13ms/step


array([[0.7818095 , 0.21819048]], dtype=float32)

In [86]:
#siamese_model.save("siamese_final_500000.h5")